## My Classifier test score
- Binary Classifier for MNIST: 92.89%
- Multi Class Classifier for MNIST: 93.56%
- Multi Class Classifier for CIFAR10: 51.47%

In [100]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

In [101]:
# Exercise 1-1
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html7

class MLP(nn.Module) :
    def __init__(self, in_dim, out_dim, inter_dim=28*28) :
        super().__init__()
        self.linear1 = nn.Linear(in_dim, inter_dim, bias=True)
        self.act1 = nn.Sigmoid()
        self.linear2 = nn.Linear(inter_dim, out_dim, bias=True)
        
    def forward(self, x) :
#         assert x.size()[2]*x.size()[3] == self.in_dim, "x.size()[2]*x.size()[3] should be {}".format(self.in_dim)
        z = self.linear1(x.view(-1, 28*28))
        z = self.act1(z)
        z = self.linear2(z)
        return z

In [102]:
STUDENT_model = MLP(28*28, 1)

In [103]:
# Please change "STUDENT_model" with your NN model class's instance name
total_params = sum(p.numel() for p in STUDENT_model.parameters())

print("Number of Parameters: ", total_params)


Number of Parameters:  616225


In [104]:
# Normalize data with mean=0.5, std=1.0
mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (1.0,))
])

# download path 정의
download_root = './MNIST_DATASET'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

In [105]:
'''
Choose two numbers and prepare datasets with two numbers.
'''
nb1 = 3
nb2 = 8

train_dataset.data = torch.cat([train_dataset.data[(train_dataset.targets == nb1)],train_dataset.data[(train_dataset.targets == nb2)]],dim=0)
train_dataset.targets = torch.cat([train_dataset.targets[(train_dataset.targets == nb1)],train_dataset.targets[(train_dataset.targets == nb2)]],dim=0)

train_dataset.targets[(train_dataset.targets == nb1)] = 0
train_dataset.targets[(train_dataset.targets == nb2)] = 1

'''
Do the same thing for the test, valid dataset
'''
valid_dataset.data = torch.cat([valid_dataset.data[(valid_dataset.targets == nb1)],valid_dataset.data[(valid_dataset.targets == nb2)]],dim=0)
valid_dataset.targets = torch.cat([valid_dataset.targets[(valid_dataset.targets == nb1)],valid_dataset.targets[(valid_dataset.targets == nb2)]],dim=0)

valid_dataset.targets[(valid_dataset.targets == nb1)] = 0
valid_dataset.targets[(valid_dataset.targets == nb2)] = 1

test_dataset.data = torch.cat([test_dataset.data[(test_dataset.targets == nb1)],test_dataset.data[(test_dataset.targets == nb2)]],dim=0)
test_dataset.targets = torch.cat([test_dataset.targets[(test_dataset.targets == nb1)],test_dataset.targets[(test_dataset.targets == nb2)]],dim=0)

test_dataset.targets[(test_dataset.targets == nb1)] = 0
test_dataset.targets[(test_dataset.targets == nb2)] = 1

In [106]:
# option 값 정의
batch_size = 8
dim = 28*28
learning_rate = 1e-3

train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [107]:
my_loss = nn.BCELoss()

In [108]:
optimizer = optim.SGD(STUDENT_model.parameters(), learning_rate)

for epoch in range(12):
    STUDENT_model.train()
    epoch_loss = 0
    
    for i, (data, target) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        output = STUDENT_model(data)
        
        output = output.reshape(-1)
        act = nn.Sigmoid()
        loss = my_loss(act(output), target.float())
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f'Epoch: {epoch} | Train Loss: {epoch_loss / len(train_loader)}')

Epoch: 0 | Train Loss: 0.6738684247030275
Epoch: 1 | Train Loss: 0.6212979668927925
Epoch: 2 | Train Loss: 0.5603490012669277
Epoch: 3 | Train Loss: 0.4907511255650399
Epoch: 4 | Train Loss: 0.4230224959084762
Epoch: 5 | Train Loss: 0.36610348859878983
Epoch: 6 | Train Loss: 0.3217530560459648
Epoch: 7 | Train Loss: 0.28818034931261327
Epoch: 8 | Train Loss: 0.26200699022857943
Epoch: 9 | Train Loss: 0.24151882196270497
Epoch: 10 | Train Loss: 0.22519464202931472
Epoch: 11 | Train Loss: 0.21188988823826624


In [109]:
### Test for train_dataset
count = 0
for i in range(train_dataset.targets.size()[0]):
    image = train_dataset.data[i]
    label = train_dataset.targets[i]

    prediction = (STUDENT_model(image.float().view(-1, 28*28)) >= torch.FloatTensor([0.5]))
    if prediction == label:
        count += 1

print('total: {}, correctly matched: {}, accuracy: {:.2f}%\n'.format(train_dataset.targets.size()[0], count, count/train_dataset.targets.size()[0] * 100. ) )

total: 11982, correctly matched: 10952, accuracy: 91.40%



In [110]:
''' Test for test_dataset '''
count = 0
wrong_answer = []
STUDENT_model.eval()
for i in range(test_dataset.targets.size()[0]):
    image = test_dataset.data[i]
    label = test_dataset.targets[i]

    prediction = STUDENT_model.forward(image.float().view(-1, 28*28)) >= torch.FloatTensor([0.5])
    if prediction == label:
        count += 1
    else:
        wrong_answer += [i]

print('total: {}, correctly matched: {} accuracy: {:.2f}%\n'.format(test_dataset.targets.size()[0], count, count/test_dataset.targets.size()[0] * 100. ) )

total: 1984, correctly matched: 1843 accuracy: 92.89%



In [111]:
# Exercise 1-2
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html
class MultiClassMLP(nn.Module) :
    def __init__(self, in_dim, out_dim, inter_dim=24*24) :
        super().__init__()
        self.in_dim = in_dim
        self.linear1 = nn.Linear(in_dim, inter_dim, bias=True)
        self.act1 = nn.Sigmoid()
        self.linear2 = nn.Linear(inter_dim, inter_dim, bias=True)
        self.act2 = nn.Sigmoid()
        self.linear3 = nn.Linear(inter_dim, out_dim, bias=True)
        
    def forward(self, x) :
        # assert x.size()[2]*x.size()[3] == self.in_dim, "x.size()[2]*x.size()[3] should be {}".format(self.in_dim)
        x = x.reshape(-1, self.in_dim)
        z = self.linear1(x)
        z = self.act1(z)
        z = self.linear2(z)
        z = self.act2(z)
        z = self.linear3(z)
        return z

In [112]:
# MultiClass Classfication
STUDENT_model = MultiClassMLP(28*28, 10)

In [113]:
# Please change "STUDENT_model" with your NN model class's instance name
total_params = sum(p.numel() for p in STUDENT_model.parameters())

print("Number of Parameters: ", total_params)

Number of Parameters:  790282


In [114]:
train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

In [115]:
# option 값 정의
batch_size = 32
dim = 28*28
learning_rate = 1e-1

train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [116]:
my_loss = nn.CrossEntropyLoss()

print(my_loss(torch.Tensor([1, 2, 3]), torch.Tensor([4, 5, 6])))
print(my_loss(torch.Tensor([1, 2, 3]), torch.Tensor([4, 2, 3])))

tensor(19.1141)
tensor(13.6685)


In [117]:
optimizer = optim.SGD(STUDENT_model.parameters(), learning_rate)

for epoch in range(7):
    STUDENT_model.train()
    epoch_loss = 0
    
    for i, (data, target) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        output = STUDENT_model(data)
        
        loss = my_loss(output, target)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f'Epoch: {epoch} | Train Loss: {epoch_loss / len(train_loader)}')

Epoch: 0 | Train Loss: 1.3938548517942428
Epoch: 1 | Train Loss: 0.4375531750837962
Epoch: 2 | Train Loss: 0.36056686911781627
Epoch: 3 | Train Loss: 0.3234294385254383
Epoch: 4 | Train Loss: 0.29063264476656914
Epoch: 5 | Train Loss: 0.2649251809378465
Epoch: 6 | Train Loss: 0.2405255453189214


In [118]:
''' Test for train_dataset '''

image_size = 28*28
#image_size = 3*32*32 # for CIFAR dataset
STUDENT_model.eval()
batch_size = 32

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
count = 0
for image,label in train_loader :
    label = label
    prediction = torch.argmax(STUDENT_model(image.view(len(image),image_size)),dim=1)
    count += (prediction == label).sum()
        
print('Test for train_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(train_dataset.targets), count/len(train_dataset.targets) * 100. ) )    

''' Test for test_dataset '''
count = 0
wrong_answer = []
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
for image,label in test_loader :
    label = label
    prediction = torch.argmax(STUDENT_model(image.view(len(image),image_size)),dim=1)
    count += (prediction == label).sum()
    
#     else:
#         wrong_answer += [i]   
        
print('Test for test_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(test_dataset.targets), count/len(test_dataset.targets) * 100. ) )    

Test for train_dataset: correctly matched/total: 56176/60000 accuracy: 93.63%

Test for test_dataset: correctly matched/total: 9356/10000 accuracy: 93.56%



In [119]:
# Exercise 1-3
class MLP_CIFAR10(nn.Module) :
    def __init__(self, in_dim, out_dim, device, inter_dim=16*16):
        super().__init__()
        self.in_dim = in_dim
        self.linear1 = nn.Linear(in_dim, inter_dim, bias=True, device=device)
        self.act1 = nn.ReLU()
        self.linear2 = nn.Linear(inter_dim, inter_dim, bias=True, device=device)
        self.act2 = nn.ReLU()
        self.linear3 = nn.Linear(inter_dim, inter_dim, bias=True, device=device)
        self.act3 = nn.ReLU()
        self.linear4 = nn.Linear(inter_dim, out_dim, bias=True, device=device)
        
    def forward(self, x) :
        # assert x.size()[2]*x.size()[3] == self.in_dim, "x.size()[2]*x.size()[3] should be {}".format(self.in_dim)
        x = x.reshape(-1, self.in_dim)
        z = self.linear1(x)
        z = self.act1(z)
        z = self.linear2(z)
        z = self.act2(z)
        z = self.linear3(z)
        z = self.act3(z)
        z = self.linear4(z)
        return z

In [120]:
from torchvision.datasets import CIFAR10

# download path 정의
download_root = './CIFAR10_DATASET'

train_dataset = CIFAR10(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = CIFAR10(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = CIFAR10(download_root, transform=mnist_transform, train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [121]:
# option 값 정의
batch_size = 32
dim = 3*32*32
learning_rate = 1e-1

train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [122]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

STUDENT_model = MLP_CIFAR10(dim, 10, device)

In [123]:
# Please change "STUDENT_model" with your NN model class's instance name
total_params = sum(p.numel() for p in STUDENT_model.parameters())

print("Number of Parameters: ", total_params)

Number of Parameters:  920842


In [124]:
my_loss = nn.CrossEntropyLoss()

In [125]:
for i, (data, target) in enumerate(train_loader):
    print(data.shape, target.shape)
    if i==0:
        break

torch.Size([32, 3, 32, 32]) torch.Size([32])


In [126]:
optimizer = optim.SGD(STUDENT_model.parameters(), learning_rate)

for epoch in range(5):
    STUDENT_model.train()
    epoch_loss = 0
    
    for i, (data, target) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        output = STUDENT_model(data.to(device))
        
        loss = my_loss(output, target.to(device))
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f'Epoch: {epoch} | Train Loss: {epoch_loss / len(train_loader)}')

Epoch: 0 | Train Loss: 1.7463650283947711
Epoch: 1 | Train Loss: 1.4615106008331973
Epoch: 2 | Train Loss: 1.3389863130074622
Epoch: 3 | Train Loss: 1.2485914128145497
Epoch: 4 | Train Loss: 1.1711103556175988


In [127]:
''' Test for train_dataset '''

#image_size = 28*28
image_size = 3*32*32 # for CIFAR dataset
STUDENT_model.eval()
batch_size = 32

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
count = 0
for image,label in train_loader :
    label = label
    prediction = torch.argmax(STUDENT_model(image.view(len(image),image_size).to(device)),dim=1)
    count += (prediction == label.to(device)).sum()
        
print('Test for train_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(train_dataset.targets), count/len(train_dataset.targets) * 100. ) )    

''' Test for test_dataset '''
count = 0
wrong_answer = []
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
for image,label in test_loader :
    label = label
    prediction = torch.argmax(STUDENT_model(image.view(len(image),image_size).to(device)),dim=1)
    count += (prediction == label.to(device)).sum()
    
#     else:
#         wrong_answer += [i]   
        
print('Test for test_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(test_dataset.targets), count/len(test_dataset.targets) * 100. ) )    

Test for train_dataset: correctly matched/total: 30099/50000 accuracy: 60.20%

Test for test_dataset: correctly matched/total: 5147/10000 accuracy: 51.47%

